In [1]:
import polars as pl # the same as pandas just faster

import configparser
from pathlib import Path
from datetime import datetime
import sys

sys.path.append('../..') # for imports to work 
from src.old_erker_mapping import map_erker2phenopackets, write_phenopackets

### Get path to notebook from config file

In [2]:
config = configparser.ConfigParser()
config.read('../../data/config/config.cfg')
real_data = False

if real_data:
    path = Path(config.get('Paths', 'mc4r_path')) 
else:
    path = Path(config.get('Paths', 'synth_data_path'))

phenopackets_out = Path(config.get('Paths', 'phenopackets_out'))

### Set Creator Tag

In [3]:
created_by = config.get('Creator', 'creator_tag')

print(f'Creator tag: {created_by}')

Creator tag: P. Robinson, MD, D. Danis, PhD, A. Graefe, F. Rehburg


### Read data in

In [4]:
df = pl.read_csv(path)
df.head(5)

record_id,sct_422549004,sct_399423000,sct_184099003_y,sct_281053000,sct_263495000,sct_315354004,sct_364699009,sct_278844005,sct_399753006,sct_420259009,sct_767023003,sct_184305005_rd,sct_16100001,sct_16100001_source,sct_769681006_center,sct_440377005,sct_276239002_aim,sct_424850005,sct_405795006_y,sct_405795006_m,sct_405795006_d,sct_423493009,sct_432213005_y,sct_432213005_m,sct_432213005_d,sct_717800004,sct_412726003,sct_64245008,sct_125679009_proposit,sct_842009,sct_439401001_orpha,sct_439401001_orpha_sub,sct_439401001_icd10gm,sct_439401001_alphaid,sct_439401001_alphaidstr,sct_439401001_bodysite,…,sct_82101005_10_age,sct_82101005_10_gender,sct_75226009_rd,sct_75226009_1_age,sct_75226009_1_gender,sct_75226009_2_age,sct_75226009_2_gender,sct_75226009_3_age,sct_75226009_3_gender,sct_75226009_4_age,sct_75226009_4_gender,sct_75226009_5_age,sct_75226009_5_gender,sct_75226009_6_age,sct_75226009_6_gender,sct_75226009_7_age,sct_75226009_7_gender,sct_75226009_8_age,sct_75226009_8_gender,sct_75226009_9_age,sct_75226009_9_gender,sct_75226009_10_age,sct_75226009_10_gender,sct_309370004_research,sct_309370004_narse,sct_441898007_data,sct_441898007_eu,sct_441898007_int,sct_441898007_aff,sct_441898007_gen,sct_441898007_case,sct_123038009,sct_840566006,sct_840566006_specific,sct_21134002_class,sct_21134002_score,erker_v15_complete
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""123AB""",null,2004,null,"""sct_248152002""",null,"""sct_372148003_…","""CS_MII_Person_…",null,null,"""sct_263659003""","""sct_1220561009…",null,null,"""2009-15-11""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_261665006""",2009,3,19,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
2,"""234BC""",null,2001,null,"""sct_248153007""",null,"""sct_90027003""","""CS_MII_Person_…",null,null,"""sct_263659003""","""sct_1220561009…",null,null,"""2011-10-21""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_410672004""",2009,7,24,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_373067005""","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
3,"""345CD""",null,2002,null,"""sct_248152002""",null,"""sct_90027003""","""CS_MII_Person_…",null,null,"""sct_41847000""","""sct_1220561009…",null,null,"""2004-5-21""","""sct_1220561009…",null,"""sct_261665006""",null,null,null,"""sct_410672004""",null,null,null,"""sct_1220561009…",null,"""sct_1220561009…","""sct_1220561009…","""sct_373067005""","""ORPHA:71529""",null,"""E66.8""","""sct_1220561009…",null,null,…,null,null,"""sct_1220561009…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…","""sct_1220561009…",null,"""sct_1220561009…",null,null
4,"""456DE""",null,2005,null,"

## Preprocessing
- remove null colus
- clean up data
- add id col to data (col name `'mc4r_id'`)

In [ ]:
# Todo
df.head(5)

## Map to phenopackets

In [7]:
phenopackets = map_erker2phenopackets(df, created_by=created_by)

TypeError: Timestamp() takes no arguments

## Write to json

In [ ]:
cur_time = datetime.now().strftime("%Y%m%d-%H%M%S")
phenopackets_out_dir = phenopackets_out / cur_time

write_phenopackets(phenopackets, phenopackets_out_dir)